<div align="center">
  <a href="https://github.com/AmberLee2427/microlens-submit">
    <img src="../assets/rges-pit_logo.png" alt="logo" width="300"/>
  </a>
</div>

# <font face="Helvetica" size="7"> Submission Tool Tutorial </font>  

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

This comprehensive tutorial demonstrates how to use [`microlens-submit`](https://microlens-submit.readthedocs.io/en/latest/) for managing your microlensing data challenge submission. Whether you're working on the Roman Research Nexus or your local machine, this guide will walk you through the complete workflow.

## What You'll Learn

- Setting up your submission project
- Adding and managing solutions for multiple events
- Recording computational metadata and provenance
- Handling degenerate solutions and model comparison
- Exporting your final submission
- Best practices for reproducible research

## Prerequisites

This tutorial assumes you have:
- Python 3.8 or higher
- `microlens-submit` installed (`pip install microlens-submit`)
- Basic familiarity with microlensing modeling


## 1. Installation and Setup

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;">

First, let's ensure we have the latest version installed:

In [ ]:
# Install or upgrade microlens-submit
!pip install --upgrade microlens-submit

In [ ]:
# Verify installation
!microlens-submit --version

## 2. Project Initialization

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

Let's create a new submission project. You can initialize it either via CLI or Python API:

In [ ]:
# Method 1: Using the CLI
!microlens-submit init --team-name "Planet Hunters" --tier "advanced"

In [ ]:
# Method 2: Using the Python API
import microlens_submit
from pathlib import Path

# Create a new submission project
project_path = Path("./my_microlensing_submission")
submission = microlens_submit.load(project_path)

# Set team information
submission.team_name = "Planet Hunters"
submission.tier = "advanced"

# Add hardware information (useful for reproducibility)
submission.hardware_info = {
    "platform": "Roman Research Nexus",
    "cpu_type": "Intel Xeon",
    "ram_gb": 32,
    "python_version": "3.10.0"
}

# Save the initial configuration
submission.save()

print(f"✅ Project initialized at: {project_path}")
print(f"Team: {submission.team_name}")
print(f"Tier: {submission.tier}")

## 3. Adding Solutions to Events

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

Now let's add solutions for different microlensing events. We'll demonstrate both simple and complex scenarios:

In [ ]:
# Get or create an event
event_001 = submission.get_event("KMT-2025-BLG-001")

# Add a simple single-lens solution
single_lens_params = {
    "t0": 2459123.5,      # Time of closest approach (JD)
    "u0": 0.15,          # Impact parameter
    "tE": 20.5,          # Einstein crossing time (days)
    "I0": 18.2,          # Baseline magnitude
    "Is": 19.8           # Source magnitude
}

solution_1 = event_001.add_solution(
    model_type="single_lens",
    parameters=single_lens_params
)

# Add metadata about this solution
solution_1.notes = "Initial single-lens fit using MCMC sampling"
solution_1.used_astrometry = False
solution_1.used_postage_stamps = True
solution_1.log_likelihood = -1234.56
solution_1.n_data_points = 1250
solution_1.lightcurve_plot_path = "plots/event_001_lc.png"
solution_1.lens_plane_plot_path = "plots/event_001_lens_plane.png"

# Record computational information
solution_1.set_compute_info(
    cpu_hours=2.5,
    wall_time_hours=0.5
)

print(f"✅ Added solution: {solution_1.solution_id}")
print(f"Model type: {solution_1.model_type}")
print(f"Parameters: {solution_1.parameters}")

In [ ]:
# Add a more complex binary-lens solution
binary_lens_params = {
    "t0": 2459123.5,
    "u0": 0.12,
    "tE": 22.1,
    "q": 0.001,           # Mass ratio
    "s": 1.15,           # Separation
    "alpha": 45.2,       # Source trajectory angle
    "rho": 0.001,        # Source size
    "I0": 18.2,
    "Is": 19.8
}

solution_2 = event_001.add_solution(
    model_type="binary_lens",
    parameters=binary_lens_params
)

# Add physical parameters derived from the fit
solution_2.physical_parameters = {
    "M_L": 0.45,         # Lens mass (M☉)
    "D_L": 6.2,          # Lens distance (kpc)
    "M_planet": 1.5,     # Planet mass (M⊕)
    "a": 2.8             # Semi-major axis (AU)
}

# Add parameter uncertainties
solution_2.parameter_uncertainties = {
    "t0": 0.1,
    "u0": 0.02,
    "tE": 0.5,
    "q": 0.0002,
    "s": 0.05
}

solution_2.notes = "Binary-lens fit with planetary companion. MCMC with 1000 walkers, 5000 steps."
solution_2.log_likelihood = -1189.34
solution_2.log_prior = -15.67
solution_2.n_data_points = 1250
solution_2.lightcurve_plot_path = "plots/event_001_lc.png"
solution_2.lens_plane_plot_path = "plots/event_001_lens_plane.png"

# Record more detailed compute info
solution_2.set_compute_info(
    cpu_hours=15.2,
    wall_time_hours=3.8
)

print(f"✅ Added binary solution: {solution_2.solution_id}")
print(f"Log-likelihood improvement: {solution_1.log_likelihood - solution_2.log_likelihood:.2f}")

## 4. Managing Multiple Events

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

Let's add solutions for additional events to demonstrate the multi-event workflow:

In [ ]:
# Add a second event with different characteristics
event_002 = submission.get_event("OGLE-2025-BLG-0042")

# This event has a more complex light curve
complex_params = {
    "t0": 2459156.2,
    "u0": 0.08,
    "tE": 35.7,
    "q": 0.0005,
    "s": 0.95,
    "alpha": 78.3,
    "rho": 0.002,
    "I0": 17.8,
    "Is": 19.2
}

solution_3 = event_002.add_solution(
    model_type="binary_lens",
    parameters=complex_params
)

solution_3.physical_parameters = {
    "M_L": 0.32,
    "D_L": 7.8,
    "M_planet": 0.8,
    "a": 1.9
}

solution_3.notes = "Complex binary event with caustic crossing. Used emcee with 2000 walkers."
solution_3.log_likelihood = -2156.78
solution_3.n_data_points = 2100
solution_3.lightcurve_plot_path = "plots/event_002_lc.png"
solution_3.lens_plane_plot_path = "plots/event_002_lens_plane.png"

solution_3.set_compute_info(cpu_hours=28.5, wall_time_hours=7.2)

print(f"✅ Added solution for {event_002.event_id}: {solution_3.solution_id}")

## 5. Solution Management and Comparison

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

Let's explore how to manage and compare different solutions:

In [ ]:
# List all solutions for an event
print("=== Solutions for KMT-2025-BLG-001 ===")
for solution in event_001.get_active_solutions():
    print(f"ID: {solution.solution_id[:8]}...")
    print(f"  Model: {solution.model_type}")
    print(f"  Log-likelihood: {solution.log_likelihood}")
    print(f"  Active: {solution.is_active}")
    print()

In [ ]:
# Compare solutions using the CLI
!microlens-submit compare-solutions KMT-2025-BLG-001

In [ ]:
# Deactivate a solution that didn't work out well
# (This keeps it in history but excludes it from final export)
solution_1.deactivate()
print(f"Deactivated solution: {solution_1.solution_id[:8]}...")
print(f"Active solutions remaining: {len(event_001.get_active_solutions())}")

## 6. Working with Posterior Samples

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

If you have posterior samples from your MCMC runs, you can store references to them:

In [ ]:
# Add posterior path to a solution
solution_2.posterior_path = "posteriors/kmt_001_binary_samples.h5"
solution_3.posterior_path = "posteriors/ogle_042_complex_samples.h5"

print(f"Posterior samples stored for: {solution_2.solution_id[:8]}...")
print(f"Path: {solution_2.posterior_path}")

## 7. Saving and Loading Progress

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

The submission state is automatically saved, but you can also save explicitly:

In [ ]:
# Save current state
submission.save()
print("✅ Project state saved")

# Reload the project (useful for long-running analyses)
reloaded_submission = microlens_submit.load(project_path)
print(f"✅ Reloaded project with {len(reloaded_submission.events)} events")

# Check that our solutions are still there
for event_id, event in reloaded_submission.events.items():
    print(f"Event {event_id}: {len(event.solutions)} solutions")

## 8. CLI Workflow Example

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

Here's how to accomplish the same tasks using the command-line interface:

In [ ]:
# Initialize project via CLI
!microlens-submit init --team-name "CLI Example Team" --tier "intermediate"

For quick validation, you can run `add-solution` with `--dry-run` to see the parsed input and schema output without saving anything.

In [ ]:
!microlens-submit add-solution KMT-2025-BLG-001 \
    --model-type single_lens \
    --param t0=2459123.5 \
    --param u0=0.15 \
    --param tE=20.5 \
    --log-likelihood -1234.56 \
    --lightcurve-plot-path plots/kmt001_lc.png \
    --lens-plane-plot-path plots/kmt001_lens.png \
    --dry-run

In [ ]:
# Add a solution via CLI
!microlens-submit add-solution KMT-2025-BLG-001 \
    --model-type single_lens \
    --param t0=2459123.5 \
    --param u0=0.15 \
    --param tE=20.5 \
    --log-likelihood -1234.56 \
    --lightcurve-plot-path plots/kmt001_lc.png \
    --lens-plane-plot-path plots/kmt001_lens.png \
    --notes "CLI example solution"

In [ ]:
# List solutions
!microlens-submit list-solutions KMT-2025-BLG-001

## 9. Final Export and Validation

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

When you're ready to submit, export your final package:

In [ ]:
# Export the final submission
submission.export(filename="planet_hunters_final_submission.zip")
print("✅ Final submission exported!")

# Check what's included
import zipfile
with zipfile.ZipFile("planet_hunters_final_submission.zip", 'r') as zip_ref:
    print("\nContents of submission package:")
    for file in zip_ref.namelist():
        print(f"  {file}")

In [ ]:
# Alternative: Export via CLI
!microlens-submit export --output cli_example_submission.zip

## 10. Best Practices and Tips

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

### Reproducibility
- Always use `set_compute_info()` to record computational details
- Include version information for key dependencies
- Use descriptive notes for each solution

### Workflow Management
- Save frequently with `submission.save()`
- Use `deactivate()` instead of deleting solutions
- Keep multiple solutions for comparison

### Data Quality
- Validate your parameters before adding solutions
- Include uncertainties when available
- Record the number of data points used

### Performance
- The tool is designed for long-term projects
- Large numbers of solutions are handled efficiently
- Export only when ready for final submission

## Next Steps

<hr style="border: 1.5pt solid #a859e4; width: 100%; margin-top: -10px;"> 

1. **Explore the API documentation** for advanced features
2. **Check the manual submission format** in `SUBMISSION_MANUAL.md` if needed
3. **Join the community** for questions and support
4. **Start your own microlensing analysis** with confidence!

Happy modeling! 🪐